In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [2]:
slider_dataset = widgets.SelectionSlider(
    options=['islands', 'moons', 'circles', 'swiss_roll'],
    value='islands',
    description='Dataset',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

slider_codebooks = widgets.SelectionSlider(
    options=[1, 2, 4, 8, 16, 32, 64, 128, 256, 512],
    value=32,
    description='No. codebook vectors',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

In [3]:
def on_value_change1(change):
    global dataset_name
    dataset_name = change['new']
slider_dataset.observe(on_value_change1, names='value')
display(slider_dataset)

SelectionSlider(continuous_update=False, description='Dataset', options=('islands', 'moons', 'circles', 'swiss…

In [4]:
def on_value_change2(change):
    global num_codebooks
    num_codebooks = change['new']
slider_codebooks.observe(on_value_change2, names='value')
display(slider_codebooks)

SelectionSlider(continuous_update=False, description='No. codebook vectors', index=5, options=(1, 2, 4, 8, 16,…

In [29]:
data = np.load(f'{dataset_name}.npy')
num_iterations = 15
data_dim = data.shape[1]

permuted_indices = np.random.permutation(data.shape[0])
codebooks = data[permuted_indices[0:num_codebooks]]
print('codebook shape = ', codebooks.shape)

codebook shape =  (16, 2)


In [30]:
expanded_data = np.expand_dims(data, axis=2)
codebook_list = []

for iter in range(num_iterations):
        new_codebooks = codebooks.copy()
        codebook_list.append(new_codebooks)
        expanded_codebooks = np.expand_dims(codebooks.T, axis=0)
        euclidean_distances = np.sum(np.square(expanded_data - expanded_codebooks), axis=1)
        best_codebooks_indices = np.argmin(euclidean_distances, axis=1)

        for i in range(codebooks.shape[0]): # codebook update step
            indices = np.where(best_codebooks_indices == i)[0]
            mean_assigned_codebooks = np.mean(data[indices], axis=0, keepdims=True)
            codebooks[i] = mean_assigned_codebooks.copy()

In [31]:
fig, ax = plt.subplots()

line1 = ax.scatter([],[])
line2 = ax.scatter([],[],marker='X', c='red')
ax.set_xlim(data[:,0].min()-1, data[:,0].max()+1)
ax.set_ylim(data[:,1].min()-1, data[:,1].max()+1)
plt.close()

def animate(frame_num):
    line1.set_offsets(data)
    line2.set_offsets(codebook_list[frame_num])
    return (line1, line2)

anim = FuncAnimation(fig, animate, frames=len(codebook_list), interval=500)

from IPython.display import HTML
HTML(anim.to_jshtml())